*Требования*:
*   оберните весь конвейер преобразований в Pipeline
*   подберите оптимальный вариант прогнозной модели с помощью GridSearchCV
*   примените обученный на тренировочных данных конвейер к тестовым данным, никак не предобрабатывая их, а лишь загрузив из файла и отделив целевой признак от остальных
*   получите на тестовой части качество не ниже 0.87 по метрике ROCAUC
Пояснение: если пропуски в новых данных и будут, то только в тех колонках, где они есть в тренировочной части.

1. Подгрузим данные